In [4]:
import open3d as o3d

# Replace 'your_file.ply' with the path to your .ply file
ply_file_path = 'luma.ply'

# Load the point cloud from the .ply file
point_cloud = o3d.io.read_point_cloud(ply_file_path)
colmap_path = "360_extra_scenes/treehill/sparse/0"


In [ ]:
from splat.read_colmap import read_images_text, qvec2rotmat
from splat.gaussians import Gaussians
from splat.gaussian_scene import GaussianScene
import pycolmap
import torch 
import torch.nn as nn
import numpy as np

from splat.utils import read_images_text, read_images_binary
import open3d as o3d

# Replace 'your_file.ply' with the path to your .ply file
ply_file_path = 'luma.ply'
point_cloud = o3d.io.read_point_cloud(ply_file_path)
colmap_path = "360_extra_scenes/treehill/sparse/0"

reconstruction = pycolmap.Reconstruction(colmap_path)
test = False

points3d = reconstruction.points3D
images = read_images_binary(f"{colmap_path}/images.bin")
cameras = reconstruction.cameras

image_num = 100
image = images[image_num]

gaussians = Gaussians(
    torch.Tensor(point_cloud.points), 
    torch.Tensor(point_cloud.colors),
    model_path="./point_clouds"
)

In [10]:
dir(point_cloud)

['HalfEdgeTriangleMesh',
 'Image',
 'LineSet',
 'PointCloud',
 'RGBDImage',
 'TetraMesh',
 'TriangleMesh',
 'Type',
 'Unspecified',
 'VoxelGrid',
 '__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'cluster_dbscan',
 'colors',
 'compute_convex_hull',
 'compute_mahalanobis_distance',
 'compute_mean_and_covariance',
 'compute_nearest_neighbor_distance',
 'compute_point_cloud_distance',
 'covariances',
 'create_from_depth_image',
 'create_from_rgbd_image',
 'crop',
 'detect_planar_patches',
 'dimension',
 'estimate_covariances',
 'estimate_normals',
 'estimate_point_covariances',
 'farthest_point_down_sample',
 'get_axis_aligned_bounding_bo

In [17]:
import numpy as np
points = np.asarray(point_cloud.points)
colors = np.asarray(point_cloud.colors)



array([-0.30839244,  0.15411164,  0.53241785])